In [1]:
import pandas as pd
import numpy as np
import joblib
from scipy.sparse import csr_matrix

In [2]:

X = joblib.load('X_preprocessed.pkl')
model_pipeline = joblib.load('anime_recommendation_model.pkl')

data = pd.read_csv('anime_filtered.csv')

genre_dummies = data['genre'].str.get_dummies(sep=', ')
data = pd.concat([data, genre_dummies], axis=1)

# Drop the original 'genre' column
data = data.drop(columns=['genre'])

In [3]:
# Function to create a user profile based on favorited animes
def create_user_profile(favorited_anime_ids, original_data, X_preprocessed):
    favorited_indices = original_data.index[original_data['anime_id'].isin(favorited_anime_ids)].tolist()
    favorited_features = X_preprocessed[favorited_indices]
    user_profile = np.asarray(favorited_features.mean(axis=0))
    return user_profile

# Function to get recommendations for a user
def get_recommendations_for_user(favorited_anime_ids, n_recommendations=5):
    user_profile = create_user_profile(favorited_anime_ids, data, X)
    user_profile = user_profile.reshape(1, -1)
    distances, indices = model_pipeline.named_steps['nearestneighbors'].kneighbors(user_profile, n_neighbors=n_recommendations + len(favorited_anime_ids))
    recommendations = data.iloc[indices[0]].copy()
    recommendations['distance'] = distances[0]
    recommendations = recommendations[~recommendations['anime_id'].isin(favorited_anime_ids)]  # Exclude already favorited animes
    return recommendations.head(n_recommendations)

In [4]:
# SHONEN
favorited_anime_ids = [5114,136,1735,32182] 
recommendations = get_recommendations_for_user(favorited_anime_ids, n_recommendations=10)
print(recommendations)

      anime_id                   title      title_english  \
4504        20                  Naruto             Naruto   
2604      1604  Katekyo Hitman Reborn!            Reborn!   
418      11061  Hunter x Hunter (2011)    Hunter x Hunter   
191       6702              Fairy Tail         Fairy Tail   
917        813           Dragon Ball Z      Dragon Ball Z   
5748     10033                  Toriko             Toriko   
14         269                  Bleach             Bleach   
169      30694       Dragon Ball Super  Dragon Ball Super   
3895       223             Dragon Ball        Dragon Ball   
5885        15            Eyeshield 21                NaN   

              title_japanese  \
4504                     ナルト   
2604        家庭教師ヒットマンREBORN!   
418   HUNTER×HUNTER ハンターハンター   
191                 フェアリーテイル   
917                 ドラゴンボールZ   
5748                     トリコ   
14           BLEACH - ブリーチ -   
169           ドラゴンボール超（スーパー）   
3895                 ドラゴンボール   
5885    

In [5]:
# SHOJO
favorited_anime_ids = [120,21995,37379,32729]
recommendations = get_recommendations_for_user(favorited_anime_ids, n_recommendations=10)
print(recommendations)

       anime_id                               title  \
1864        145             Kareshi Kanojo no Jijou   
1175        509            I My Me! Strawberry Eggs   
48         9656           Kimi ni Todoke 2nd Season   
8729      31109                    Hatsukoi Monster   
16         6045                      Kimi ni Todoke   
10435     18671  Chuunibyou demo Koi ga Shitai! Ren   
1699        325                          Peach Girl   
8038       1721                            Hitohira   
3869      35639                       Just Because!   
9969        667         Kashimashi: Girl Meets Girl   

                                        title_english     title_japanese  \
1864                        His and Her Circumstances            彼氏彼女の事情   
1175                         I My Me! Strawberry Eggs  あぃまぃみぃ！ストロベリー・エッグ   
48                   Kimi ni Todoke: From Me To You 2    君に届け 2ND SEASON   
8729                               First Love Monster            初恋モンスター   
16            

In [6]:
# SHOJO AND SPORT
favorited_anime_ids = [120,21995,37379,28891,22125] 
recommendations = get_recommendations_for_user(favorited_anime_ids, n_recommendations=10)
print(recommendations)

       anime_id                               title  \
7895      34636                  Ballroom e Youkoso   
8779      18507                               Free!   
5805      20583                           Haikyuu!!   
4908      22265               Free!: Eternal Summer   
10435     18671  Chuunibyou demo Koi ga Shitai! Ren   
1864        145             Kareshi Kanojo no Jijou   
48         9656           Kimi ni Todoke 2nd Season   
16         6045                      Kimi ni Todoke   
13669     10460                       Kimi to Boku.   
1175        509            I My Me! Strawberry Eggs   

                                        title_english         title_japanese  \
7895                          Welcome to the Ballroom            ボールルームへようこそ   
8779                        Free! - Iwatobi Swim Club                  Free!   
5805                                         Haikyu!!                ハイキュー!!   
4908                           Free! - Eternal Summer  Free!-Eternal Summe

In [12]:
# SHONEN AND MECHA
favorited_anime_ids = [136,2904,30,32729] 
recommendations = get_recommendations_for_user(favorited_anime_ids, n_recommendations=10)
print(recommendations)

       anime_id                            title  \
10476     35849            Darling in the FranXX   
13726      1575  Code Geass: Hangyaku no Lelouch   
6268        168                         s.CRY.ed   
996          90          Mobile Suit Gundam Wing   
2716       6213          Toaru Kagaku no Railgun   
5411        237                     Eureka Seven   
5399          6                           Trigun   
1136        302              Mirai Shounen Conan   
10918      2001       Tengen Toppa Gurren Lagann   
951       21437                    Buddy Complex   

                              title_english   title_japanese  \
10476                 DARLING in the FRANXX  ダーリン・イン・ザ・フランキス   
13726  Code Geass: Lelouch of the Rebellion  コードギアス 反逆のルルーシュ   
6268                               s-CRY-ed            スクライド   
996                 Mobile Suit Gundam Wing       新機動戦記ガンダムW   
2716           A Certain Scientific Railgun       とある科学の超電磁砲   
5411                           Eureka Seven